# Candidate/Running config database

In this demo we are going to see how jetconf let's you manage the 

## Starting things up

Run:

    make start-dev-containers

Now, let's export some environment variables we need:

In [1]:
export USER_CERT=../../tests/certs/test_user_curl.pem
export BASE_URL=https://ntc-rosetta-conf:8443

## Basics

Even though it's not part of the specification, jetconf implements a netconf-like workflow where changes are made against a candidate database and commited in a single transaction into the running database. There are a few notes to be made here.

Even though changes are always made against the candidate database, the `GET` method can be used against the running configuration. To support this the URL target will be slightly different:

* Use `$BASE_URL/restconf/data/...` to target the candidate database.
* Use `$BASE_URL/restconf_running/data/...` to target the running database.

To operate the candidata database, jetconf supports the following operational endpoints:

| Path                                           | Method | Use                                  |
|------------------------------------------------|--------|--------------------------------------|
| /restconf/operations/jetconf:conf-reset        | POST   | Discard candidate configuration      |
| /restconf/operations/jetconf:conf-commit       | POST   | commit candidate configuration       |
| /restconf/operations/jetconf:get-schema-digest | POST   | retrieve supported schema            |
| /restconf/operations/jetconf:conf-status       | POST   | retrieve status of the configuration |

Let's see a few examples, let's start by adding an interface:

In [2]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": []
    }
}

In [3]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @2_candidate_config/add_interface_eth0.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

Now, let's verify the candidate database:

In [4]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "description": "a test interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            }
        ]
    }
}

The changes are there, let's look at the running database:

In [5]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": []
    }
}

The changes are not yet present there, let's do a couple of other changes; let's add another interface:

In [6]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    -d @2_candidate_config/add_interface_eth1.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

Now, let's verify the different configuration databases:

In [7]:
# candidate:devices
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "description": "a test interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            },
            {
                "name": "eth1",
                "config": {
                    "name": "eth1",
                    "description": "another test interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            }
        ]
    }
}

In [8]:
# running:devices
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": []
    }
}

Pretty much what we expected. Now we can do three things:

- `/restconf/operations/jetconf:conf-reset` - Discard the changes
- `/restconf/operations/jetconf:conf-commit` - Commit the changes
- `restconf/operations/jetconf:conf-status` - Verify the status of the configuration

Let's start by verifying the status (should report that there is a transaction opened), commit the changes and then verify the status again (should report the transaction is not closed):

In [9]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-status

{
    "status": "OK",
    "transaction-opened": true
}

In [10]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}

In [11]:
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-status

{
    "status": "OK",
    "transaction-opened": false
}

Now, let's verify the running config:

In [12]:
# running:interfaces
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X GET \
    $BASE_URL/restconf_running/data/openconfig-interfaces:interfaces

{
    "openconfig-interfaces:interfaces": {
        "interface": [
            {
                "name": "eth0",
                "config": {
                    "name": "eth0",
                    "description": "a test interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            },
            {
                "name": "eth1",
                "config": {
                    "name": "eth1",
                    "description": "another test interface",
                    "type": "iana-if-type:ethernetCsmacd"
                }
            }
        ]
    }
}

In [13]:
# ignore me, this deletes the data so the notebook can be rerun
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X PUT \
    -d @../../tests/data/interfaces_empty.json \
    $BASE_URL/restconf/data/openconfig-interfaces:interfaces
curl --http2 -k --cert-type PEM -E $USER_CERT \
    -X POST \
    $BASE_URL/restconf/operations/jetconf:conf-commit

{
    "status": "OK",
    "conf-changed": true
}